# Homework 1 (Due Thursday, October 28th, 2021 at 6:29pm PST)


**Rubric**


Every day late is -10%.

You are a business analyst working for a major US toy retailer:

A **Product Marketing Manager** in the marketing department wants to find out the most frequently used words in positive reviews (five stars) and negative reviews (one star) in order to determine what occasion the toys are purchased for (Christmas, birthdays, and anniversaries.). She would like your opinion on **which age groups**(what ages) recipients of toys seem to have the highest reviews, and **which occasion has the most positive reviews.**

* There are **malformed characters in the review** text. For instance, notice the `&#34;` - these are examples of incorrectly decoded [HTML encodings](https://krypted.com/utilities/html-encoding-reference/).
```
"amazing quality first of all, these cards are amazing proxies (but don't try to use em in &#34;official duels&#34; unless a judge is okay with it, if you have the real thing to show) and look amazing in your binder!"
```
Please clean up all instances of these incorrect decodings. 

Perform the same word count analysis using the reviews received from Amazon to answer your marketing manager's question. They are stored in two files, (`poor_amazon_toy_reviews.txt`) and (`good-amazon-toy-reviews.txt`). **Provide a few sentences with your findings and business recommendations.** Make any assumptions you'd like to- this is a fictitious company after all. I just want you to get into the habit of "finishing" your analysis: to avoid delivering technical numbers to a non-technical manager.

Some considerations in your analysis:

* Use **regular expressions to parse out all references to recipients and gift occassions**, and account for the possibility that people may spell words "son" / "children" / "4 year old" as both singular and plural, upper or lower-cased.

* Explain what some of **pitfalls/limitations** are of using only a word count analysis to make these inferences. What additional research/steps would you need to do to verify your conclusions?

**Submit everything as a new notebook and Slack direct message to me (Yu Chen) and the TA the HW as an attachment.**

**NOTE**: Name the notebook `lastname_firstname_HW1.ipynb`.

In [1]:
import sys
import re
import pandas as pd

In [2]:
positive_text = open("good_amazon_toy_reviews.txt")
print(positive_text)
positive_text.readline()

<_io.TextIOWrapper name='good_amazon_toy_reviews.txt' mode='r' encoding='UTF-8'>


'Excellent!!!\n'

In [3]:
#how big is the file? 
positive_text.seek(0)
lines = positive_text.readlines()
print(f'This positive reviews file has {len(lines)} lines')

This positive reviews file has 102217 lines


In [4]:
#code taken from lecture
from collections import Counter

def count_words(lines, delimiter=" "):
    
    words = Counter() # instantiate a Counter object called words
    for line in lines:
        for word in line.split(delimiter):
            words[word] += 1 # increment count for word
    return words

count_words(lines)

Counter({'Excellent!!!\n': 10,
         '"Great': 1381,
         'quality': 3740,
         'wooden': 255,
         'track': 369,
         '(better': 3,
         'than': 5549,
         'some': 3889,
         'others': 470,
         'we': 6175,
         'have': 14375,
         'tried).': 2,
         'Perfect': 1674,
         'match': 231,
         'to': 63339,
         'the': 96606,
         'various': 226,
         'vintages': 1,
         'of': 36593,
         'Thomas': 153,
         'that': 18143,
         'already': 764,
         'have.': 148,
         'There': 1241,
         'is': 42321,
         'enough': 2094,
         'here': 650,
         'fun': 6507,
         'and': 86557,
         'get': 6040,
         'creative': 232,
         'incorporating': 5,
         'your': 4349,
         'key': 140,
         'pieces': 1925,
         'with': 28829,
         'splits,': 3,
         'loops': 31,
         'bends."\n': 1,
         'my': 28718,
         'daughter': 5522,
         'loved': 7196

# I) Occasions Analysis

In [5]:
#we want to remove occasions of &#34
pos_df = pd.DataFrame(open("good_amazon_toy_reviews.txt", "r"), columns = ["line"])
pos_df["line"] = pos_df["line"].str.replace("\n", "")
pos_df["line"] = pos_df["line"].str.replace("&#34;", "")
count_words(pos_df["line"])
count_words(pos_df["line"]).most_common()

[('the', 96630),
 ('and', 86557),
 ('a', 66623),
 ('to', 63361),
 ('for', 48413),
 ('I', 48403),
 ('', 45008),
 ('it', 44364),
 ('is', 42328),
 ('of', 36599),
 ('this', 32780),
 ('with', 28860),
 ('my', 28721),
 ('in', 25576),
 ('was', 20102),
 ('are', 18651),
 ('that', 18153),
 ('on', 17827),
 ('as', 16965),
 ('so', 14814),
 ('but', 14697),
 ('The', 14480),
 ('have', 14401),
 ('you', 14091),
 ('My', 13609),
 ('great', 13168),
 ('loves', 12579),
 ('love', 12363),
 ('very', 11817),
 ('they', 10224),
 ('It', 10192),
 ('be', 9567),
 ('one', 9122),
 ('little', 9033),
 ('not', 8893),
 ('all', 8880),
 ('old', 8876),
 ('Great', 8746),
 ('can', 8709),
 ('them', 8526),
 ('like', 8435),
 ('at', 8385),
 ('it.', 8226),
 ('just', 8164),
 ('This', 8151),
 ('has', 7762),
 ('/><br', 7715),
 ('year', 7612),
 ('these', 7526),
 ('her', 7465),
 ('will', 7248),
 ('loved', 7231),
 ('or', 7138),
 ('up', 6884),
 ('fun', 6777),
 ('good', 6688),
 ('he', 6515),
 ('really', 6513),
 ('kids', 6274),
 ('had', 6255),

### We have now removed instances of \n and &#34

**The next steps will focus on targeting age groups and occasions**

- We will first find all reviews that revolve around **occasions**
- We will then do word counts on these filtered reviews to see which occasions have the most positive reviews
- We will look into our non-occasions and see if that frame has any occasions we missed
- We finally look into negative reviews and occasions

In [6]:
pos_df.line.str
pos_df["line"].str.findall(r'christmas').sort_values(ascending=False)

40699    [christmas, christmas]
24605    [christmas, christmas]
25182    [christmas, christmas]
95783    [christmas, christmas]
25747    [christmas, christmas]
                  ...          
68134                        []
68135                        []
68136                        []
68137                        []
0                            []
Name: line, Length: 102217, dtype: object

In [7]:
# find all occasions
pos_df["occasion"] = pos_df["line"].str.findall(r'(birthday|birthdays|Bday|bday|Birthday|\
|Halloween|halloween|Halloweens|\
present|presents|Present|Presents|gift|gifts|Gift|Gifts|\
Christmas|christmas|Easter|easter|\
anniversay|Annivesary|wedding|\
graduation|Graduation)')
pos_df.sort_values("occasion", ascending = False)

,line,occasion
9026,The gloves were perfect. I had a motorcycle t...,"[wedding, wedding]"
62537,Everybody loved these props. I used them at a...,"[wedding, wedding]"
2670,"""Bought this for my under one year old nephew,...","[wedding, wedding]"
48261,"""Used this paired with the A Kiss and We're Of...","[wedding, wedding]"
52289,Purchased this for our daughters wedding .. be...,"[wedding, wedding]"
...,...,...
35638,Great product!,[]
35636,Loved the product. It exceeded my expectations...,[]
35635,I like it a lot. It's small but great size for...,[]
35634,6 year old daughter loves the swing! Very easy...,[]


### Filtered occasions

- We now have a dataframe with each review and corresponding column occasion indicating if the review has an occasion. If a column has [] then it does not correspond to an occasion we have defined
- We will now filter our data on occasion and get word counts based on these occasions
- We will also have the dataframe with [] and will check to make sure we aren't missing any occasions

In [8]:
pos_occasions = pos_df.loc[pos_df["occasion"].str.len() != 0]
pos_no_occasions = pos_df.loc[pos_df["occasion"].str.len() == 0]
count_words(pos_occasions["line"]).most_common()
#count_words(pos_no_occasions["line"]).most_common()
# run commented out line of code to see if there are any occasions we missed

[('the', 13231),
 ('and', 13079),
 ('a', 12488),
 ('for', 10971),
 ('to', 9142),
 ('I', 7113),
 ('it', 6848),
 ('', 6500),
 ('is', 5714),
 ('this', 5597),
 ('my', 5491),
 ('of', 5057),
 ('was', 4267),
 ('with', 4134),
 ('as', 3628),
 ('in', 3563),
 ('gift', 3513),
 ('are', 2557),
 ('on', 2531),
 ('that', 2519),
 ('birthday', 2502),
 ('so', 2367),
 ('The', 2054),
 ('her', 1916),
 ('but', 1842),
 ('It', 1780),
 ('have', 1774),
 ('great', 1731),
 ('be', 1686),
 ('she', 1631),
 ('they', 1622),
 ('he', 1608),
 ('his', 1589),
 ('loves', 1555),
 ('you', 1526),
 ('very', 1482),
 ('at', 1476),
 ('loved', 1470),
 ('little', 1423),
 ('one', 1383),
 ('year', 1377),
 ('This', 1370),
 ('them', 1363),
 ('old', 1336),
 ('it.', 1298),
 ('these', 1270),
 ('will', 1266),
 ('My', 1253),
 ('/><br', 1239),
 ('love', 1216),
 ('all', 1213),
 ('not', 1180),
 ('can', 1167),
 ('or', 1160),
 ('has', 1128),
 ('bought', 1121),
 ('just', 1095),
 ('up', 1029),
 ('He', 1014),
 ('like', 1013),
 ('had', 998),
 ('would',

### Occasions in positive reviews

- **For our occasions filered dataframe:** In order, gifts, birthdays, parties, Christmas, and Halloween seem to be the occasions with the most 5-star reviews
- **For our non-occasions filered dataframe:** We ran the most common words and did not see any pertinent occasions. This means that it seems we did a good job filering on occasions

We now run the same analysis of occasions on negative reviews

In [9]:
negative_text = open("poor_amazon_toy_reviews.txt")
lines = negative_text.readlines()
print(f'This negative reviews file has {len(lines)} lines')
neg_df = pd.DataFrame(open("poor_amazon_toy_reviews.txt", "r"), columns = ["line"])
neg_df["line"] = neg_df["line"].str.replace("\n", "")
neg_df["line"] = neg_df["line"].str.replace("&#34;", "")
count_words(neg_df["line"]).most_common()

This negative reviews file has 12700 lines


[('the', 18998),
 ('and', 10910),
 ('I', 9630),
 ('to', 9591),
 ('a', 9204),
 ('it', 8891),
 ('', 7361),
 ('of', 6692),
 ('not', 5462),
 ('is', 5449),
 ('was', 5344),
 ('for', 4785),
 ('this', 4371),
 ('in', 4132),
 ('with', 3137),
 ('that', 2961),
 ('on', 2956),
 ('my', 2878),
 ('The', 2866),
 ('but', 2774),
 ('are', 2400),
 ('have', 2214),
 ('as', 2086),
 ('you', 1978),
 ('one', 1971),
 ('very', 1825),
 ('so', 1820),
 ('they', 1780),
 ('be', 1754),
 ('It', 1658),
 ('at', 1622),
 ('had', 1604),
 ('would', 1564),
 ('were', 1523),
 ('like', 1459),
 ('all', 1374),
 ('out', 1304),
 ('just', 1271),
 ('get', 1262),
 ('them', 1228),
 ('even', 1180),
 ('from', 1173),
 ('when', 1165),
 ('only', 1163),
 ('This', 1151),
 ('up', 1142),
 ('we', 1126),
 ('or', 1104),
 ('it.', 1020),
 ('will', 1001),
 ('got', 978),
 ('no', 972),
 ('after', 934),
 ('what', 903),
 ('product', 883),
 ('these', 879),
 ('if', 871),
 ('buy', 859),
 ('your', 852),
 ("didn't", 852),
 ('because', 850),
 ('did', 848),
 ('off'

In [10]:
# find all occasions
neg_df["occasion"] = neg_df["line"].str.findall(r'(birthday|birthdays|Bday|bday|Birthday|\
|Halloween|halloween|Halloweens|\
present|presents|Present|Presents|gift|gifts|Gift|Gifts|\
Christmas|christmas|Easter|easter|\
anniversay|Annivesary|wedding|\
graduation|Graduation)')
neg_df.sort_values("occasion", ascending = False)
neg_occasions = neg_df.loc[neg_df["occasion"].str.len() != 0]
count_words(neg_occasions["line"]).most_common()

[('the', 1899),
 ('a', 1312),
 ('and', 1311),
 ('it', 1157),
 ('to', 1142),
 ('I', 1108),
 ('for', 905),
 ('', 844),
 ('was', 737),
 ('of', 708),
 ('this', 616),
 ('my', 550),
 ('is', 515),
 ('not', 468),
 ('in', 426),
 ('as', 382),
 ('that', 364),
 ('on', 337),
 ('with', 317),
 ('birthday', 301),
 ('but', 300),
 ('The', 281),
 ('have', 263),
 ('gift', 230),
 ('so', 225),
 ('one', 215),
 ('had', 207),
 ('be', 202),
 ('we', 199),
 ('were', 197),
 ('would', 196),
 ('are', 191),
 ('very', 187),
 ('they', 184),
 ('you', 183),
 ('at', 178),
 ('It', 173),
 ('his', 160),
 ('all', 154),
 ('bought', 141),
 ('out', 133),
 ('when', 131),
 ('up', 131),
 ('because', 130),
 ('them', 130),
 ('got', 128),
 ('like', 127),
 ('get', 126),
 ('even', 122),
 ('from', 120),
 ('just', 120),
 ('no', 116),
 ('This', 115),
 ('We', 113),
 ('son', 111),
 ('he', 111),
 ('her', 111),
 ('these', 110),
 ('only', 109),
 ('will', 106),
 ('or', 102),
 ('My', 101),
 ('year', 100),
 ('an', 100),
 ('old', 99),
 ('received',

### Occasions in negative reviews

- **For our negative reviews:** Our occasions with the most 1-star ratings are birthdays and gifts. We will look into this more when we analyze age groups and give recommendaions

# II) Age Analysis

In [11]:
#find all ages
#daughter, Children, grandson, 7-yr, children, Grandson, grandchildren's, grandchildren, preschooler
# Granddaughter, kids, son, daughter
#yr, year-old, x-year,kids,kid

In [12]:
pos_df["line"].str.findall(r'-yr').sort_values(ascending=False)

51921    [-yr, -yr]
27545    [-yr, -yr]
43786         [-yr]
36161         [-yr]
52897         [-yr]
            ...    
68141            []
68142            []
68143            []
68144            []
0                []
Name: line, Length: 102217, dtype: object

In [13]:
pos_df = pd.DataFrame(open("good_amazon_toy_reviews.txt", "r"), columns = ["line"])
pos_df["line"] = pos_df["line"].str.replace("\n", "")
pos_df["line"] = pos_df["line"].str.replace("&#34;", "")
count_words(pos_df["line"]).most_common()

[('the', 96630),
 ('and', 86557),
 ('a', 66623),
 ('to', 63361),
 ('for', 48413),
 ('I', 48403),
 ('', 45008),
 ('it', 44364),
 ('is', 42328),
 ('of', 36599),
 ('this', 32780),
 ('with', 28860),
 ('my', 28721),
 ('in', 25576),
 ('was', 20102),
 ('are', 18651),
 ('that', 18153),
 ('on', 17827),
 ('as', 16965),
 ('so', 14814),
 ('but', 14697),
 ('The', 14480),
 ('have', 14401),
 ('you', 14091),
 ('My', 13609),
 ('great', 13168),
 ('loves', 12579),
 ('love', 12363),
 ('very', 11817),
 ('they', 10224),
 ('It', 10192),
 ('be', 9567),
 ('one', 9122),
 ('little', 9033),
 ('not', 8893),
 ('all', 8880),
 ('old', 8876),
 ('Great', 8746),
 ('can', 8709),
 ('them', 8526),
 ('like', 8435),
 ('at', 8385),
 ('it.', 8226),
 ('just', 8164),
 ('This', 8151),
 ('has', 7762),
 ('/><br', 7715),
 ('year', 7612),
 ('these', 7526),
 ('her', 7465),
 ('will', 7248),
 ('loved', 7231),
 ('or', 7138),
 ('up', 6884),
 ('fun', 6777),
 ('good', 6688),
 ('he', 6515),
 ('really', 6513),
 ('kids', 6274),
 ('had', 6255),

In [14]:
pos_df["age"] = pos_df["line"].str.findall(r'(-YR|-yr|-year|-YEAR| year old\
old|OLD|\
son|SON|Son|Sons|SONS|\
daughter|DAUGHTER|daughters|DAUGHTERS|Daughters|\
children|CHILDREN|Children|child|CHILD|Child|\
[1-18] y| -y| -Y)')

pos_df.sort_values("age", ascending = False)
pos_age = pos_df.loc[pos_df["age"].str.len() != 0]
pos_age = pos_df.loc[pos_df["age"].str.len() == 0] 
count_words(pos_age["line"]).most_common()

[('the', 81389),
 ('and', 73271),
 ('a', 56144),
 ('to', 52842),
 ('I', 41700),
 ('for', 41028),
 ('it', 38524),
 ('', 38227),
 ('is', 35453),
 ('of', 30793),
 ('this', 28048),
 ('my', 24660),
 ('with', 24127),
 ('in', 21695),
 ('was', 17701),
 ('are', 15485),
 ('on', 15203),
 ('that', 14619),
 ('as', 14601),
 ('but', 12696),
 ('so', 12622),
 ('My', 12177),
 ('The', 12124),
 ('have', 12016),
 ('you', 11965),
 ('great', 11923),
 ('loves', 11519),
 ('love', 10835),
 ('very', 10320),
 ('It', 8654),
 ('Great', 8367),
 ('they', 8315),
 ('one', 7895),
 ('be', 7834),
 ('little', 7758),
 ('old', 7749),
 ('all', 7563),
 ('not', 7361),
 ('it.', 7359),
 ('like', 7241),
 ('them', 7168),
 ('just', 7065),
 ('at', 7064),
 ('can', 7030),
 ('This', 6992),
 ('year', 6816),
 ('loved', 6570),
 ('has', 6497),
 ('these', 6406),
 ('her', 6316),
 ('good', 6131),
 ('will', 5927),
 ('/><br', 5812),
 ('fun', 5804),
 ('or', 5771),
 ('up', 5705),
 ('he', 5663),
 ('really', 5632),
 ('kids', 5467),
 ('had', 5350),
 

# Positive Reviews: Age

- **Based on our parsing** The ages with the most positive reviews are, in order, 2, 3, 5, 4, 6, 8, 10, and 7. This includes occurences of words (two for 2) 

We will now do the same thing for negative reviews

In [15]:
neg_df = pd.DataFrame(open("poor_amazon_toy_reviews.txt", "r"), columns = ["line"])
neg_df["line"] = neg_df["line"].str.replace("\n", "")
neg_df["line"] = neg_df["line"].str.replace("&#34;", "")
neg_df["age"] = neg_df["line"].str.findall(r'(-YR|-yr|-year|-YEAR| year old\
old|OLD|\
son|SON|Son|Sons|SONS|\
daughter|DAUGHTER|daughters|DAUGHTERS|Daughters|\
children|CHILDREN|Children|child|CHILD|Child|\
[1-18] y| -y| -Y)')

neg_age = neg_df.loc[neg_df["age"].str.len() != 0]
#neg_age = pos_df.loc[neg_df["age"].str.len() == 0] 
count_words(neg_age["line"]).most_common()

[('the', 1820),
 ('to', 1090),
 ('and', 1067),
 ('a', 987),
 ('I', 874),
 ('', 842),
 ('it', 757),
 ('of', 661),
 ('for', 595),
 ('is', 546),
 ('this', 491),
 ('not', 455),
 ('was', 417),
 ('in', 417),
 ('that', 375),
 ('with', 365),
 ('my', 350),
 ('on', 298),
 ('are', 270),
 ('The', 264),
 ('but', 258),
 ('have', 241),
 ('you', 216),
 ('be', 216),
 ('child', 198),
 ('as', 191),
 ('would', 184),
 ('very', 177),
 ('they', 174),
 ('so', 169),
 ('at', 148),
 ('will', 147),
 ('It', 140),
 ('your', 139),
 ('were', 138),
 ('one', 138),
 ('or', 137),
 ('like', 132),
 ('had', 131),
 ('we', 127),
 ('them', 126),
 ('from', 126),
 ('just', 126),
 ('all', 124),
 ('out', 123),
 ('it.', 117),
 ('children', 117),
 ('even', 116),
 ('up', 111),
 ('if', 110),
 ('when', 105),
 ('/><br', 104),
 ('an', 103),
 ('no', 103),
 ('get', 101),
 ('these', 98),
 ('toy', 96),
 ('This', 95),
 ('about', 95),
 ('only', 95),
 ('has', 91),
 ('because', 87),
 ('could', 86),
 ('use', 82),
 ('My', 82),
 ('too', 79),
 ("don

### For negative reviews, it also seems like 2 and 3 year olds are the most common but this could be explained by the fact that 2/3 year olds are buying the toy the most!

**Now, let's put everything together and give recommendations!**

In [16]:
pos_df = pd.DataFrame(open("good_amazon_toy_reviews.txt", "r"), columns = ["line"])
pos_df["line"] = pos_df["line"].str.replace("\n", "")
pos_df["line"] = pos_df["line"].str.replace("&#34;", "")

pos_df["parse"] = pos_df["line"].str.findall(r'(birthday|birthdays|Bday|bday|Birthday|\
|Halloween|halloween|Halloweens|\
present|presents|Present|Presents|gift|gifts|Gift|Gifts|\
Christmas|christmas|Easter|easter|\
anniversary|Annivesary|wedding|\
graduation|Graduation|\
-YR|-yr|-year|-YEAR| year old\
old|OLD|\
son|SON|Son|Sons|SONS|\
daughter|DAUGHTER|daughters|DAUGHTERS|Daughters|\
children|CHILDREN|Children|child|CHILD|Child|\
[1-18] y| -y| -Y|[1-18]y)')

pos_helper = pos_df.loc[pos_df["parse"].str.len() != 0]
count_words(pos_helper["line"]).most_common()

[('the', 25573),
 ('and', 23644),
 ('a', 20560),
 ('to', 17593),
 ('for', 16405),
 ('I', 12391),
 ('', 11991),
 ('it', 11432),
 ('is', 11218),
 ('of', 9758),
 ('this', 9208),
 ('my', 8590),
 ('with', 7934),
 ('in', 6647),
 ('was', 6013),
 ('that', 5414),
 ('as', 5394),
 ('are', 5085),
 ('on', 4601),
 ('so', 4049),
 ('The', 3918),
 ('have', 3741),
 ('gift', 3513),
 ('but', 3482),
 ('you', 3238),
 ('they', 3119),
 ('be', 3034),
 ('It', 2952),
 ('/><br', 2759),
 ('her', 2746),
 ('great', 2669),
 ('very', 2667),
 ('My', 2521),
 ('can', 2505),
 ('birthday', 2502),
 ('love', 2497),
 ('at', 2495),
 ('loves', 2457),
 ('them', 2436),
 ('not', 2431),
 ('little', 2407),
 ('one', 2363),
 ('old', 2281),
 ('This', 2280),
 ('will', 2280),
 ('all', 2264),
 ('he', 2249),
 ('she', 2207),
 ('or', 2206),
 ('has', 2146),
 ('these', 2135),
 ('his', 2107),
 ('year', 1997),
 ('like', 1996),
 ('loved', 1976),
 ('it.', 1971),
 ('up', 1971),
 ('just', 1963),
 ('we', 1832),
 ('would', 1750),
 ('when', 1746),
 ('h

In [17]:
# and we do the same thing for negative reviews
neg_df = pd.DataFrame(open("poor_amazon_toy_reviews.txt", "r"), columns = ["line"])
neg_df["line"] = neg_df["line"].str.replace("\n", "")
neg_df["line"] = neg_df["line"].str.replace("&#34;", "")

neg_df["parse"] = neg_df["line"].str.findall(r'(birthday|birthdays|Bday|bday|Birthday|\
|Halloween|halloween|Halloweens|\
present|presents|Present|Presents|gift|gifts|Gift|Gifts|\
Christmas|christmas|Easter|easter|\
anniversay|Annivesary|wedding|\
graduation|Graduation|\
-YR|-yr|-year|-YEAR| year old\
old|OLD|\
son|SON|Son|Sons|SONS|\
daughter|DAUGHTER|daughters|DAUGHTERS|Daughters|\
children|CHILDREN|Children|child|CHILD|Child|\
[1-18] y| -y| -Y|[1-18]y)')

neg_helper = neg_df.loc[neg_df["parse"].str.len() != 0]
count_words(neg_helper["line"]).most_common()

[('the', 3427),
 ('and', 2144),
 ('a', 2068),
 ('to', 2026),
 ('I', 1834),
 ('it', 1745),
 ('', 1509),
 ('for', 1350),
 ('of', 1231),
 ('was', 1048),
 ('this', 1013),
 ('is', 969),
 ('not', 848),
 ('my', 824),
 ('in', 760),
 ('that', 668),
 ('with', 630),
 ('on', 566),
 ('but', 518),
 ('as', 513),
 ('The', 504),
 ('have', 463),
 ('are', 430),
 ('be', 386),
 ('so', 365),
 ('you', 359),
 ('they', 345),
 ('very', 334),
 ('would', 332),
 ('one', 314),
 ('were', 312),
 ('had', 306),
 ('birthday', 301),
 ('we', 297),
 ('at', 293),
 ('It', 290),
 ('all', 250),
 ('like', 243),
 ('out', 235),
 ('them', 234),
 ('will', 231),
 ('gift', 230),
 ('just', 230),
 ('up', 224),
 ('when', 224),
 ('from', 223),
 ('even', 220),
 ('or', 210),
 ('no', 202),
 ('get', 202),
 ('bought', 201),
 ('because', 199),
 ('child', 198),
 ('it.', 198),
 ('This', 192),
 ('only', 191),
 ('his', 191),
 ('these', 191),
 ('if', 185),
 ('your', 185),
 ('an', 175),
 ('got', 172),
 ('We', 170),
 ('My', 166),
 ('could', 165),
 ('

## Final Recommendations for Age and Occasion

- **Positive Reviews:**
    - Birthdays for 2/3 year old children and kids seem to have the most 5-star reviews
    - Parties and 4/5 year olds also have quite a few 5-star reviews
    - If our manager wanted to target a secondary market, Christmas, grandchildren, 7, 8 and 10 year olds could be the focus, although the first 2 bullet points are the most common
    
### =>  Our product marketing manager should target birthday parties for kids aged 2-5

- **Pitfalls & Limitations:**
    - Although much less frequent, ages are still coming up in our non-parsed frame
    - Besides knowing that the reviews were either 1/5 star, we don't assess what the reviews are actually saying
    - To verify our conclusions about ages, we would need to make sure the numbers we are capturing ALL refer to ages.
    - To verify our conclusions about occasions, we could look at date fields around holidays and if we had more data about birthdays we could relate the review data to a birthday.